# LanceDB Index Demo
In this notebook we are going to show how to use [LanceDB](https://www.lancedb.com) to perform vector searches in LlamaIndex

In [5]:
import logging
import sys

# Uncomment to see debug logs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from gpt_index import SimpleDirectoryReader, Document, StorageContext
from gpt_index.indices.vector_store import GPTVectorStoreIndex
from gpt_index.vector_stores import LanceDBVectorStore
import textwrap

### Setup OpenAI
The first step is to configure the openai key. It will be used to created embeddings for the documents loaded into the index

In [3]:
import os
os.environ['OPENAI_API_KEY'] = ""

### Loading documents
Load the documents stored in the `paul_graham_essay/data` using the SimpleDirectoryReader

In [6]:
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()
print('Document ID:', documents[0].doc_id, 'Document Hash:', documents[0].doc_hash)

Document ID: 540fb252-c98d-46b3-98f1-06b55bcbd335 Document Hash: 77ae91ab542f3abb308c4d7c77c9bc4c9ad0ccd63144802b7cbe7e1bb3a4094e


### Create the index
Here we create an index backed by LanceDB using the documents loaded previously. GPTLanceDBIndex takes a few arguments.
- uri (str, required): Location where LanceDB will store its files.
- table_name (str, optional): The table name where the embeddings will be stored. Defaults to "vectors".
- nprobes (int, optional): The number of probes used. A higher number makes search more accurate but also slower. Defaults to 20.
- refine_factor: (int, optional): Refine the results by reading extra elements and re-ranking them in memory. Defaults to None

- More details can be found at the [LanceDB docs](https://lancedb.github.io/lancedb/ann_indexes)

In [7]:
vector_store = LanceDBVectorStore(uri='/tmp/lancedb')
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context)

INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 20729 tokens


### Query the index
We can now ask questions using our index.

In [9]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is the author?")

INFO:gpt_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 5 tokens
INFO:gpt_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1828 tokens
INFO:gpt_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


In [10]:
print(textwrap.fill(str(response), 100))

 The author is Paul Graham.


In [11]:
response = query_engine.query("What did the author do growing up?")

INFO:gpt_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 8 tokens
INFO:gpt_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1906 tokens
INFO:gpt_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


In [12]:
print(textwrap.fill(str(response), 100))

 The author grew up writing short stories and programming on the IBM 1401. He also wrote simple
games, a program to predict how high his model rockets would fly, and a word processor on a TRS-80.
He initially planned to study philosophy in college, but eventually switched to AI.


### Appending data
You can also add data to an existing index

In [13]:
del index

index = GPTVectorStoreIndex.from_documents([Document("The sky is blue")], uri="/tmp/new_dataset")

INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 4 tokens


In [14]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is the author?")
print(textwrap.fill(str(response), 100))

INFO:gpt_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 5 tokens
INFO:gpt_index.token_counter.token_counter:> [get_response] Total LLM token usage: 43 tokens
INFO:gpt_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


 The author is unknown.


In [15]:
index = GPTVectorStoreIndex.from_documents(documents, uri="/tmp/new_dataset")

INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 20729 tokens


In [17]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is the author?")
print(textwrap.fill(str(response), 100))

INFO:gpt_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 5 tokens
INFO:gpt_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1823 tokens
INFO:gpt_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


 The author is Paul Graham.
